<a href="https://colab.research.google.com/github/gomyk/NNStudy/blob/moonwon/%5BRL%5D%5BQN%5D%5BMW%5DCartPole-V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40)

import math
import glob
import io
import base64
from IPython.display import HTML

from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400,900),)
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

HIDDEN = 64
BATCH_SIZE = 32
PERCENTILE = 50
GAMMA = 0.9
NUM_EPISODES = 200

class Net(nn.Module):
  def __init__(self, obs_size, hidden, action_size):
    super(Net, self).__init__()
    self.net=nn.Sequential(
        nn.Linear(obs_size, hidden),
        nn.ReLU(),
        nn.BatchNorm1d(hidden),
        nn.Linear(hidden, action_size)
    )

  def forward(self, x):
    return self.net(x)

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    for mp4 in mp4list:
      video = io.open(mp4, 'r+b').read()
      print(mp4)
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 300px;">
                                  <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
                                  </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")


In [10]:
env = gym.wrappers.Monitor(gym.make("CartPole-v1"), "video", force=True, video_callable=lambda c:c%100 ==0)
net = Net(env.observation_space.shape[0], HIDDEN, env.action_space.n)
net.eval()
criterion = nn.MSELoss()
opt = optim.Adam(net.parameters(), 0.001)

for i in range(NUM_EPISODES):
  e = 1. / ((i/10) + 1)
  step_count = 0
  state = env.reset()
  done = False
  total_loss = 0
  while not done:
    step_count +=1 
    state_tensor = torch.FloatTensor([state])
    Qs = net(state_tensor).data
    origin_Qs = net(state_tensor)
    if np.random.rand(1) < e:
      action = env.action_space.sample()
    else:
      action = np.argmax(Qs.numpy())
  
    next_state , reward, done, _ = env.step(action)
    if done:
      Qs[0, action] = -100 
      print('%d episode  step : %d  loss : %.2f'%(i, step_count, total_loss/step_count))
    else:
      next_state_tensor = torch.FloatTensor([next_state])
      Qs1 = net(next_state_tensor)
      Qs[0, action] = reward + (GAMMA * np.max(Qs1.data.numpy()))
      loss = criterion(Qs, origin_Qs)
      total_loss += loss
      opt.zero_grad()
      loss.backward()
      opt.step()
      state = next_state
    



0 episode  step : 41  loss : 0.55
1 episode  step : 35  loss : 0.52
2 episode  step : 12  loss : 0.49
3 episode  step : 17  loss : 0.52
4 episode  step : 8  loss : 0.49
5 episode  step : 16  loss : 0.60
6 episode  step : 20  loss : 0.51
7 episode  step : 12  loss : 0.53
8 episode  step : 11  loss : 0.46
9 episode  step : 19  loss : 0.61
10 episode  step : 11  loss : 0.70
11 episode  step : 12  loss : 1.09
12 episode  step : 10  loss : 0.93
13 episode  step : 10  loss : 1.14
14 episode  step : 10  loss : 1.47
15 episode  step : 16  loss : 0.93
16 episode  step : 9  loss : 0.58
17 episode  step : 15  loss : 4.07
18 episode  step : 10  loss : 3.81
19 episode  step : 13  loss : 4.53
20 episode  step : 35  loss : 6.15
21 episode  step : 11  loss : 4.02
22 episode  step : 13  loss : 2.39
23 episode  step : 9  loss : 10.17
24 episode  step : 9  loss : 1.00
25 episode  step : 9  loss : 14.73
26 episode  step : 11  loss : 24.71
27 episode  step : 10  loss : 0.99
28 episode  step : 11  loss : 2.